In [ ]:
# test if videos are processed properly

In [64]:
import pandas as pd
labels_df = pd.read_csv("datasets/data/labels_complete.csv", index_col=False)
labels_df.head()

,Unnamed: 0,camera_index,video_folder,video_index,stage,start,end,Task Name,Env Descriptions: light,Env Descriptions: drawer,Env Descriptions: cloth,Env Descriptions: oven
0,0,0,draw_cloth_light,0,0,0.00,12.13,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed
1,1,0,draw_cloth_light,0,1,12.13,27.71,cloth,the light is off,the drawer is closed,the cloth is not in the sink,the oven door is closed
2,2,0,draw_cloth_light,0,2,27.71,40.43,light,the light is off,the drawer is closed,the cloth is in the sink,the oven door is closed
3,3,0,draw_cloth_light,1,0,0.00,13.60,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed
4,4,0,draw_cloth_light,1,1,13.60,32.78,cloth,the light is off,the drawer is closed,the cloth is not in the sink,the oven door is closed


In [14]:
import zarr
import numpy as np

zarrfile = zarr.open("datasets/data_v2/draw_cloth_light/replay_buffer.zarr", mode="r")
np.array(zarrfile['data']['action'])

(23684, 7)

In [15]:
zarrfile.tree()

AttributeError: 'Tree' object has no attribute '_ipython_display_'

/
 ├── data
 │   ├── action (23684, 7) float64
 │   ├── gripper_force (23684,) float64
 │   ├── gripper_pos (23684,) float64
 │   ├── gripper_state (23684,) int64
 │   ├── robot_eef_pose (23684, 6) float64
 │   ├── robot_eef_pose_vel (23684, 6) float64
 │   ├── robot_joint (23684, 6) float64
 │   ├── robot_joint_vel (23684, 6) float64
 │   ├── stage (23684,) int64
 │   └── timestamp (23684,) float64
 └── meta
     └── episode_ends (25,) int64

In [18]:
zarrhuman = zarr.open("datasets/data_v2/human_draw_cloth_light/replay_buffer.zarr", mode="r")
zarrfile.tree()
#np.array(zarrfile['data']['action'])

AttributeError: 'Tree' object has no attribute '_ipython_display_'

/
 ├── data
 │   ├── action (23684, 7) float64
 │   ├── gripper_force (23684,) float64
 │   ├── gripper_pos (23684,) float64
 │   ├── gripper_state (23684,) int64
 │   ├── robot_eef_pose (23684, 6) float64
 │   ├── robot_eef_pose_vel (23684, 6) float64
 │   ├── robot_joint (23684, 6) float64
 │   ├── robot_joint_vel (23684, 6) float64
 │   ├── stage (23684,) int64
 │   └── timestamp (23684,) float64
 └── meta
     └── episode_ends (25,) int64

In [1]:
import json

with open("datasets/config/data.json", "r") as f:
    json_data = json.load(f)

def replace_numbers_with_minus_one(obj):
    if isinstance(obj, int):
        return -1
    elif isinstance(obj, list):
        return [replace_numbers_with_minus_one(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: replace_numbers_with_minus_one(value) for key, value in obj.items()}
    else:
        return obj
    
json_data = replace_numbers_with_minus_one(json_data)

with open("datasets/config/data.json", "w") as f:
    json.dump(json_data, f)


In [28]:
import pandas as pd
import numpy as np

dat = pd.read_csv("datasets/timestamps.csv")
def func(g):
    '''
    :param g: a tuple representing the group of video_folder and video_index
              e.g. g = ('draw_cloth_light', 0)
    '''
    
    return [1, 2, 3]

dat.groupby(['video_folder', 'video_index'])['start'].apply(lambda g: [g])

video_folder      video_index
draw_cloth_light  0              [[0.0, 12.13, 27.71]]
                  1               [[0.0, 13.6, 32.78]]
                  2              [[0.0, 14.67, 27.14]]
                  3               [[0.0, 12.22, 29.2]]
                  4               [[0.0, 9.73, 24.68]]
                  5              [[0.0, 11.05, 26.45]]
                  6                [[0.0, 10.1, 25.8]]
                  7              [[0.0, 10.33, 23.17]]
                  8               [[0.0, 8.52, 24.56]]
                  9               [[0.0, 9.01, 21.33]]
Name: start, dtype: object

In [61]:
import json
import pandas as pd
import numpy as np

dat = pd.read_csv("datasets/timestamps.csv")
with open("datasets/config/data.json") as f:
    metadata = json.load(f)
'''
dat.groupby(['video_folder', 'video_index']).apply(lambda x: 
                                                   metadata['data'][video_folder])
'''

# generates task names and append them to dat
dat['stage'] = dat.groupby(['video_folder', 'video_index']).cumcount()
def func(row):
    v_folder = row['video_folder']
    stage_idx = row['stage']
    return metadata['data'][v_folder]['stages'][stage_idx]
dat['Task Name'] = dat.apply(func, axis=1)

# generate env descriptions
def get_env_descriptions(video_folder, task_name):
    env_descriptions_binary = metadata['data'][video_folder]['Env Descriptions'][task_name]
    descriptions_map = metadata['Env Descriptions']
    return [descriptions_map[i][b] for i, b in enumerate(env_descriptions_binary)]

env_d = dat.apply(lambda row: get_env_descriptions(row['video_folder'], row['Task Name']),
          axis=1)
env_d = env_d.apply(pd.Series)

colnames = [f"Env Descriptions: {obj}" for obj in metadata["Env_Objects"]]
env_d.columns = colnames 

dat = pd.concat([dat, env_d], axis=1)


In [62]:
dat

,camera_index,video_folder,video_index,stage,start,end,Task Name,Env Descriptions: light,Env Descriptions: drawer,Env Descriptions: cloth,Env Descriptions: oven
0,0,draw_cloth_light,0,0,0.00,12.13,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed
1,0,draw_cloth_light,0,1,12.13,27.71,cloth,the light is off,the drawer is closed,the cloth is not in the sink,the oven door is closed
2,0,draw_cloth_light,0,2,27.71,40.43,light,the light is off,the drawer is closed,the cloth is in the sink,the oven door is closed
3,0,draw_cloth_light,1,0,0.00,13.60,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed
4,0,draw_cloth_light,1,1,13.60,32.78,cloth,the light is off,the drawer is closed,the cloth is not in the sink,the oven door is closed
5,0,draw_cloth_light,1,2,32.78,43.33,light,the light is off,the drawer is closed,the cloth is in the sink,the oven door is closed
6,0,draw_cloth_light,2,0,0.00,14.67,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed
7,0,draw_cloth_light,2,1,14.67,27.14,cloth,the light is off,the drawer is closed,the cloth is not in the sink,the oven door is closed
8,0,draw_cloth_light,2,2,27.14,38.70,light,the light is off,the drawer is closed,the cloth is in the sink,the oven door is closed
9,0,draw_cloth_light,3,0,0.00,12.22,draw,the light is off,the drawer is open,the cloth is not in the sink,the oven door is closed


In [50]:
get_env_descriptions("draw_cloth_light", "light")

['the light is off',
 'the drawer is closed',
 'the cloth is in the sink',
 'the oven door is closed']

In [41]:
def custom_function(video_folder, stage):
    # Your logic here, for example, printing the parameters
    print(f"Video Folder: {video_folder}, Video Index: {stage}")

# Apply the custom function to each group defined by 'video_folder' and 'stage'
dat.groupby(['video_folder', 'video_index']).apply(lambda group: custom_function(group['video_folder'].iloc[0], group['video_index'].iloc[0]))


Video Folder: draw_cloth_light, Video Index: 0
Video Folder: draw_cloth_light, Video Index: 1
Video Folder: draw_cloth_light, Video Index: 2
Video Folder: draw_cloth_light, Video Index: 3
Video Folder: draw_cloth_light, Video Index: 4
Video Folder: draw_cloth_light, Video Index: 5
Video Folder: draw_cloth_light, Video Index: 6
Video Folder: draw_cloth_light, Video Index: 7
Video Folder: draw_cloth_light, Video Index: 8
Video Folder: draw_cloth_light, Video Index: 9


""
